# Experiments

> Where the set of experiments and the single experiments are defined.

In [ ]:
#| default_exp experiments

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti

from sklearn.base import BaseEstimator
from typing import List, Dict, Tuple

## Experiment
One single expleriment defined

In [ ]:
#| export

class Experiment:

    def __init__(self,
        detector: str,
        embeddingModel: str,
        scorer: BaseEstimator,
        calibrator: BaseEstimator,
        calibration_db: List[str],
        enfsi_years: List[int],
        filters: List[str],
        face_image_filters: List[str],
        metrics: str,
        n_calibration_pairs: int,
        embedding_model_as_scorer: bool,
        root_output_dir: str
        ):

        self.detector = detector
        self.embeddingModel = embeddingModel
        self.scorer = scorer
        self.calibrator = calibrator
        self.calibration_db = calibration_db
        self.enfsi_years = enfsi_years
        self.filters = filters
        self.face_image_filters = face_image_filters
        self.metrics = metrics
        self.n_calibration_pairs = n_calibration_pairs
        self.embedding_model_as_scorer = embedding_model_as_scorer
        self.root_output_dir = root_output_dir
        self.output_dir = get_output_dir()


    def get_output_dir(self):
        if isinstance(self.calibrator, IsotonicCalibrator):
            calibrator = 'Isotonic Calibrator'
        else:
            calibrator = str(self.calibrator)

        if self.embedding_model_as_scorer:
            folder = f'{self.detector}_{self.embeddingModel}(emb=scorer)_{calibrator.split("(")[0]}'
        else:
            folder = f'{self.detector}_{self.embeddingModel}(emb<>scorer)_{calibrator.split("(")[0]}'

        output_dir = os.path.join(self.root_output_dir, folder)
        return output_dir
    
    
    def perform(self) -> Dict[str, float]:
        """
        Function to run a single experiment with pipeline:
        - Fit model on train data
        - Fit calibrator on calibrator data
        - Evaluate test set
        """

        # Create folder
        self.create_output_dir()
        session = connect_db()

        # Get test pairs per category.
        test_pairs_per_category = self.get_test_pairs_per_category(session)
        # Get calibration pair per category.
        calibration_pairs_per_category = self.get_calibration_pairs_per_category(test_pairs_per_category.keys(),
                                                                                 session)
        '''
        # todo: move later.
        pairs_df = pd.DataFrame(columns=['Image 1', 'Image 2'])
        for category, pairs in calibration_pairs_per_category.items():
            print(category, len(pairs))
            for pair in pairs:
                pairs_df = pairs_df.append({'Image 1': pair.first.croppedImages.images.path, 'Image 2': pair.second.croppedImages.images.path}, ignore_index=True)

        pairs_df.to_excel(f'cal_pairs_{self.filters}_{self.face_image_filters}.xlsx')
        '''

        # Generate lr_system per category.
        lr_systems, test_pairs_per_category = self.generate_lr_systems(calibration_pairs_per_category,
                                                                       test_pairs_per_category, session)

        # Predict LR
        results = self.predict_lr(lr_systems, test_pairs_per_category, session)

        # todo: make necessary variables for graphs.
        return results

   
    def create_output_dir(self):
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)


    def __str__(self):
        """
        Converts the configuration of this experiment to a string that can be
        used to generate file names for example.
        """
        data_values = []
        for k, v in self.data_config.items():
            if k == 'datasets' and isinstance(v, tuple):
                data_values.append('|'.join(map(str, v)))
            else:
                data_values.append(str(v))

        params_str = '_'.join(map(str, self.params.values()))
        return '_'.join(map(str, [
            self.scorer,
            self.calibrator,
            params_str
        ])).replace(':', '-')  # Windows forbids ':'

        return None

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()